In [8]:
import requests
import os
from newspaper import Article

API_KEY = 'YOUR API KEY'  # <-- Replace with your key here

class CustomArticle(Article):
    def download(self):
        self.config.browser_user_agent = (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
        )
        return super().download()

def save_article(index, title, content, url, folder):
    filename = os.path.join(folder, f"article_{index}.txt")
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"Title: {title}\n\n")
        f.write(f"URL: {url}\n\n")
        f.write(f"Full Article:\n{content}\n")
    print(f"Saved: {filename}")

def fetch_news(topic, max_articles=5):#<------Your limit here
    url = (
        'https://newsapi.org/v2/everything?'
        f'q={topic}&'
        'language=en&'
        'sortBy=relevance&'
        f'apiKey={API_KEY}'
    )
    response = requests.get(url)
    data = response.json()
    if data.get('status') != 'ok':
        print("NewsAPI error:", data.get('message'))
        return []
    return data.get('articles', [])[:max_articles]

def main():
    topic = input("Enter topic to search news for: ").strip()
    articles = fetch_news(topic)

    if not articles:
        print("No articles found or error fetching news.")
        return

    folder = f"news_full_{topic.replace(' ', '_')}"
    os.makedirs(folder, exist_ok=True)

    success_count = 0
    fail_count = 0

    for i, art in enumerate(articles, start=1):
        title = art.get('title', 'No title')
        url = art.get('url')
        if not url:
            print(f"Skipping article {i}: No URL found")
            fail_count += 1
            continue

        print(f"\nProcessing article {i}: {title}")
        try:
            article = CustomArticle(url)
            article.download()
            article.parse()
            content = article.text if article.text else "Full text not available."
            save_article(i, title, content, url, folder)
            success_count += 1
        except Exception as e:
            print(f"Failed to extract article {i}: {e}")
            fail_count += 1

    print(f"\nDone! {success_count} articles saved, {fail_count} failed.")

if __name__ == "__main__":
    main()


Enter topic to search news for:  football



Processing article 1: Watch: The Women's Football Show
Saved: news_full_football\article_1.txt

Processing article 2: Monday Night Club
Saved: news_full_football\article_2.txt

Processing article 3: Fubo isn’t giving up on its Venu-like sports bundle
Saved: news_full_football\article_3.txt

Processing article 4: Palmer's brilliance could be key moment in Chelsea's Champions League quest
Saved: news_full_football\article_4.txt

Processing article 5: 'It's a huge error' - controversy as Villa miss out on Champions League
Saved: news_full_football\article_5.txt

Done! 5 articles saved, 0 failed.
